In [32]:
import re
from nltk.collocations import TrigramCollocationFinder

In [33]:

def preprocess_train(text):
    text = text.lower() 
    text = re.sub(' {2}', ' ', text)
    text = re.sub(r'\n', '  ', text)
    text = re.sub(r'[\d%,)•($:]', '', text)             
    return text

In [35]:
def find_parameters(text):
    finder = TrigramCollocationFinder.from_words(text)
    dic = {}# Las ci ocurrencias del trigrama es el valor en el diccionario
    N = 0
    for k, v in finder.ngram_fd.items(): # N supongo qu es la suma de todos los valores de las claves del diccionario 
        if v > 5:
            dic[k] = v
        N += v
    B = len(dic)
    return [B, N, dic]
    

In [36]:
from math import log

def lindstone(sentence, c_train, B_train, N_train):
    lamda = 0.05
    prob_acum = 0
    finder = TrigramCollocationFinder.from_words(sentence)

    for n in finder.ngram_fd.items():
        grama = n[0]
        freq = c_train[grama]
        prob_acum += log(freq + lamda) / (N_train + B_train * lamda)
    return prob_acum

############# hay que desacer el log posteriormente?
############# tenemos que comprobar todos los tests con todos los train de todos los idiomas? SÍ

In [37]:
Param_lang = {}

In [38]:
def train(l1, l2):
    #l1 lista de srings de nombres de ficheros (o direcciones donde se encuentra el fichero, tambien str)
    # l2 lista de nombres de idioma a la que pertenece cada fichero
    # len(l1) == len(l2)
    for i in range(len(l1)):
        with open(l1[i], 'r', encoding='utf-8') as f:
            raw_text = f.read()
        prep_text = preprocess_train(raw_text)
        Param_lang[l2[i]] = find_parameters(prep_text)
        


In [ ]:
def predict(test_files):
    #test_files lista de strings de fichero de test
    Files_classified = [None]*len(test_files)
    
    for t in range(len(test_files)):
        with open(test_files[t], 'r', encoding='utf-8') as f:
            raw_test = f.read()

        prep_test = preproces_test(raw_test)
        sentences = prep_test.split("  ") #suponiendo que funciona que cada oracion esta separada por solo 2 espacios
        D_answer = {}
        Answer = [None]*len(sentences)
        for s in range(len(sentences)):
            for k in Param_lang.keys():
                l = Param_lang[k]
                B = l[0]
                N = l[1]
                c = l[2]
                prob = lindstone(sentences[s], c, B, N)
                D_answer[k] = prob
            Answer[s] = max(D_answer, key=D_answer.get)
        Files_classified[t] = Answer

In [39]:
train_files = ['eng_trn.txt', 'spa_trn.txt', 'fra_trn.txt', 'ita_trn.txt', 'deu_trn.txt']
lang_of_train_f = ['eng', 'spa', 'fra', 'ita', 'deu']
train(train_files, lang_of_train_f)

In [ ]:
test_files =['eng_tst.txt', 'ita_tst.txt', 'fra_tst.txt', 'spa_tst.txt', 'deu_tst.txt']
Class_matrix = predict(test_files)